## Simulation in the Inventory Model ( Julia )

This code simulates the inventory of a firm many times.

It generates a matrix with `num_paths` rows and `sim_length` columns.

One simulation of inventory dynamics corresponds to one row.  

The question is: how well does naive parallelization work?

In [1]:
addprocs(3)
@everywhere using Distributions

In [2]:
@everywhere mutable struct SP
    s::Float64
    S::Float64
    p::Float64
end


@everywhere function SP()
    
    s = 10.0
    S = 100.0
    p = 0.4
    SP(s, S, p)   
end

## Sequential:

In [3]:
function sim_paths(sp::SP;initial_x=50.0, num_paths=100000,sim_length=400)
    
    X = Array{Int32}(num_paths,sim_length)
    X[:, 1] = initial_x
    
    # For each row
    for i = 1:num_paths
        
        dvals = rand(Geometric(sp.p),sim_length)
        for t = 1:sim_length-1
            x, d = X[i, t], dvals[t]
            if x <= sp.s
                y = max(sp.S - d, 0)
            else
                y = max(x - d, 0)
            end
            
            X[i, t+1] = y
            
            end
        end
       return X
    
end

sim_paths (generic function with 1 method)

In [4]:
sp = SP();

compilation time:

In [5]:
@time X = sim_paths(sp);

  3.904120 seconds (40.49 M allocations: 1.062 GiB, 9.07% gc time)


running time:

In [6]:
@time X = sim_paths(sp)

  3.541251 seconds (40.46 M allocations: 1.060 GiB, 6.86% gc time)


100000×400 Array{Int32,2}:
 50  44  43  41  37  37  35  35  34  …  25  25  25  25  25  25  25  19  19
 50  50  47  47  46  45  45  45  43     20  18  18  18  18  16  16  15  15
 50  47  47  44  44  44  39  38  38     25  22  18  17  16  15  14  11   9
 50  49  45  42  40  40  34  33  33     95  95  95  95  95  94  92  92  91
 50  43  40  33  33  33  31  23  22     53  52  51  50  49  45  45  38  38
 50  47  45  44  43  41  40  40  39  …  55  50  47  46  46  43  42  42  41
 50  49  49  46  44  40  38  37  21     25  23  23  22  22  20  20  19  18
 50  49  46  45  43  40  40  38  38     69  68  66  59  55  54  54  51  50
 50  46  44  42  41  41  41  40  36     66  65  62  59  57  51  44  43  43
 50  45  44  43  42  38  34  27  27     18  18  16  16  15  15  14  14  11
 50  49  47  46  46  45  45  45  44  …  16  14  13   6  97  89  89  89  87
 50  50  50  50  50  49  49  49  47     43  41  34  33  33  33  33  30  28
 50  46  33  33  32  31  31  27  27     77  77  76  76  76  75  75  75  7

## Parallel:

In [7]:
@everywhere function sim_paths(sp::SP;initial_x=50.0, 
                                num_paths=100000, sim_length=400)
    
    X = SharedArray{Int32}(num_paths,sim_length)
    X[:, 1] = initial_x
    
    # For each row
    @sync @parallel for i=1:num_paths
        dvals = rand(Geometric(sp.p),sim_length)
        
        for t=1:sim_length-1
            x, d = X[i, t], dvals[t]
            
            if x <= sp.s
                y = max(sp.S - d, 0)
            else
                y = max(x - d, 0)
            end
        
            X[i, t+1] = y
            
        end
    end
    
    return X  
end

compilation time:

In [8]:
@time X = sim_paths(sp);

  7.476439 seconds (1.33 M allocations: 74.709 MiB, 0.55% gc time)


running time:

In [9]:
@time X = sim_paths(sp)

  1.892505 seconds (2.49 k allocations: 126.245 KiB)


100000×400 SharedArray{Int32,2}:
 50  48  48  48  45  45  43  42  40  36  …  56  54  54  53   53   53  50  49
 50  50  47  47  43  43  43  42  42  39     34  33  28  22   22   22  20  20
 50  47  44  42  42  42  41  41  41  37     79  77  77  76   74   72  72  72
 50  48  47  45  44  44  41  40  37  36     14   7  95  94   94   94  91  89
 50  48  46  46  37  36  36  36  36  35     90  90  90  90   89   87  85  85
 50  49  46  46  43  41  33  33  29  27  …  82  79  78  78   78   77  77  77
 50  50  50  50  50  49  46  42  42  41     36  35  32  32   30   25  24  22
 50  48  47  46  46  43  41  41  41  39     40  37  34  21   20   20  19  19
 50  48  46  45  44  43  43  43  43  43     64  62  62  61   60   56  54  54
 50  50  47  46  39  39  39  39  39  39     44  41  39  39   39   38  38  33
 50  50  50  50  50  50  46  46  41  33  …  66  66  65  65   64   64  64  61
 50  48  46  44  44  40  39  36  32  30     34  34  32  31   31   23  19  19
 50  50  43  40  37  37  35  35  35  35    